# CasADi vs Theano

###### Jupyter notebook to compare the speed of Theano and CasADi softwares of automatic differentiation on the specific task of building force kernels

###### The TwoBody is fully implemented in both softwares and Theano appears to be sensibly faster. Probably this is due to the use of numpy arrays

# Two Body

## Theano 

In [1]:
import theano.tensor as T
from theano import function, scan
import numpy as np

In [2]:
# Theano functions

def compile_twobody_singlespecies():
    """
    This function generates theano compiled kernels for energy and force learning

    The position of the atoms relative to the centrla one, and their chemical species
    are defined by a matrix of dimension Mx5

    Returns:
        k2_ee (func): energy-energy kernel
        k2_ef (func): energy-force kernel
        k2_ff (func): force-force kernel
    """
    print("Started compilation of theano two body single species kernels")
    # --------------------------------------------------
    # INITIAL DEFINITIONS
    # --------------------------------------------------

    # positions of central atoms
    r1, r2 = T.dvectors('r1d', 'r2d')
    # positions of neighbours
    rho1, rho2 = T.dmatrices('rho1', 'rho2')
    # lengthscale hyperparameter
    sig = T.dscalar('sig')
    # cutoff hyperparameters
    theta = T.dscalar('theta')
    rc = T.dscalar('rc')

    # positions of neighbours without chemical species (3D space assumed)
    rho1s = rho1#[:, 0:3]
    rho2s = rho2#[:, 0:3]

    # --------------------------------------------------
    # RELATIVE DISTANCES TO CENTRAL VECTOR
    # --------------------------------------------------

    # first and second configuration
    r1j = T.sqrt(T.sum((rho1s[:, :] - r1[None, :])** 2, axis=1))
    r2m = T.sqrt(T.sum((rho2s[:, :] - r2[None, :])** 2, axis=1))

    # Cutoff function
    k_ij = T.exp(-(r1j[:, None] - r2m[None, :])** 2 /(2*sig**2))

    cut_ij = (0.5 * (1 + T.sgn(rc - r1j[:, None]))) * (0.5 * (1 + T.sgn(rc - r2m[None, :]))) * \
             (T.exp(-theta / (rc - r1j[:, None])) * T.exp(-theta / (rc - r2m[None, :])))

    k_ij = k_ij #* cut_ij

    # kernel
    k = T.sum(k_ij)

    # --------------------------------------------------
    # FINAL FUNCTIONS
    # --------------------------------------------------

    # energy energy kernel
    k_ee_fun = function([r1, r2, rho1, rho2, sig, theta, rc], k, allow_input_downcast=False, on_unused_input='ignore')

    # energy force kernel
    k_ef = T.grad(k, r2)
    k_ef_fun = function([r1, r2, rho1, rho2, sig, theta, rc], k_ef, allow_input_downcast=False, on_unused_input='ignore')

    # force force kernel
    k_ff = T.grad(k, r1)
    k_ff_der, updates = scan(lambda j, k_ff, r2: T.grad(k_ff[j], r2),
                             sequences=T.arange(k_ff.shape[0]), non_sequences=[k_ff, r2])

    k_ff_fun = function([r1, r2, rho1, rho2, sig, theta, rc], k_ff_der, allow_input_downcast=False,
                        on_unused_input='ignore')

    # --------------------------------------------------
    # WRAPPERS (we don't want to plug the position of the central element every time)
    # --------------------------------------------------

    def k2_ee(conf1, conf2, sig, theta, rc):
        """
        Two body kernel for energy-energy correlation

        Args:
            conf1: first configuration.
            conf2: second configuration.
            sig: lengthscale hyperparameter.
            theta: cutoff smoothness hyperparameter.
            rc: cutoff distance hyperparameter.

        Returns:
            kernel (scalar):

        """
        return k_ee_fun(np.zeros(3), np.zeros(3), conf1, conf2, sig, theta, rc)

    def k2_ef(conf1, conf2, sig, theta, rc):
        """
        Two body kernel for energy-force correlation

        Args:
            conf1: first configuration.
            conf2: second configuration.
            sig: lengthscale hyperparameter.
            theta: cutoff smoothness hyperparameter.
            rc: cutoff distance hyperparameter.

        Returns:
            kernel (vector):
        """

        return k_ef_fun(np.zeros(3), np.zeros(3), conf1, conf2, sig, theta, rc)

    def k2_ff(conf1, conf2, sig, theta, rc):
        """
        Two body kernel for force-force correlation

        Args:
            conf1: first configuration.
            conf2: second configuration.
            sig: lengthscale hyperparameter.
            theta: cutoff smoothness hyperparameter.
            rc: cutoff distance hyperparameter.

        Returns:
            kernel (matrix):
        """

        return k_ff_fun(np.zeros(3), np.zeros(3), conf1, conf2, sig, theta, rc)

    print("Ended compilation of theano two body single species kernels")

    return k2_ee, k2_ef, k2_ff



In [3]:
k2_ee, k2_ef, k2_ff = compile_twobody_singlespecies()

Started compilation of theano two body single species kernels
Ended compilation of theano two body single species kernels


## CasADi

In [72]:
from casadi import *

In [73]:
# fix the number of neighbours
M = 20

In [86]:
# --------------------------------------------------
# INITIAL DEFINITIONS
# --------------------------------------------------

r1, r2 = MX.sym( 'r1', 3  ), MX.sym( 'r2', 3 )
rho1, rho2 = MX.sym('rho1', 3, M), MX.sym('rho2',3,  M)
sig = MX.sym('sig')

In [88]:
# kernel
k = MX.zeros()
for i in range(M):
    for j in range(M):
        k = k + exp(-(norm_2(rho1[:, i] - r1) - norm_2(rho2[:,j] - r2))**2/(2*sig**2))

In [89]:
# single and double derivative
grad = gradient(k, r1)
hess = jacobian( grad , r2)

In [90]:
# define CasADi functions
k2_ee_c = Function('k2_ee_c', [rho1, rho2, r1, r2, sig], [k], ['rho1', 'rho2', 'r1', 'r2', 'sig'] , [ 'k' ] )
k2_ef_c = Function('k2_ef_c', [rho1, rho2, r1, r2, sig], [grad], ['rho1', 'rho2', 'r1', 'r2', 'sig'] , [ 'k_g' ] )
k2_ff_c = Function('k2_ff_c', [rho1, rho2, r1, r2, sig], [hess], ['rho1', 'rho2', 'r1', 'r2', 'sig'] , [ 'k_h' ] )

In [91]:
# generate a CasADi C-compiled function
k2_ff_c.generate('k2_ff_c.c')


'k2_ff_c.c'

In [30]:
# compile CasADi function
!gcc -shared -fPIC k2_ff_c.c -o k2_ff_c.so

In [31]:
# import C-compiled function
k2_ff_c_comp = external( 'k2_ff_c', 'k2_ff_c.so' )

## Python (numba)

In [32]:
from numba import jit

In [102]:
def gauss(d1, d2, sigma):
    return np.exp(-(d1 - d2) ** 2 / (2 * sigma ** 2))

def hessian(r1, r2, d1, d2, sigma):
    return np.outer(r1, r2) * (1 - (d1 - d2) ** 2 / sigma ** 2) / (sigma ** 2 * d1 * d2) * np.exp(-(d1 - d2) ** 2 / (2 * sigma ** 2))


def fill_gram_python(n, d, nn, confs):
    gram = np.zeros((n * d, n * d))
    for i in np.arange(n):
        for j in np.arange(n):
            tmp = np.zeros((3,3))
            for ii in np.arange(nn):
                r1 = confs[i, ii, :]
                d1 = np.linalg.norm(r1)
                for jj in np.arange(nn):
                    r2 = confs[j, jj, :]
                    d2 = np.linalg.norm(r2)
                    tmp += hessian(r1, r1, d1, d2, 1.0)
            gram[d * i:d * (i + 1), d * j:d * (j + 1)] = tmp
    return gram

@jit()
def fill_gram_python_numba(n, d, nn, confs):
    sigma = 1.0
    
    gram = np.zeros((n * d, n * d))
    dd = np.linalg.norm(confs, axis=2)
    
    for i in range(n):
        for j in range(n):
            tmp = np.zeros((3,3))
            for ii in range(nn):
                r1 = confs[i, ii, :]
                d1 = dd[i, ii]
                for jj in range(nn):
                    r2 = confs[j, jj, :]
                    d2 = dd[j, jj]
                    tmp += np.outer(r1, r2) * (1 - (d1 - d2) ** 2 / sigma ** 2) / (sigma ** 2 * d1 * d2) * np.exp(-(d1 - d2) ** 2 / (2 * sigma ** 2))

            gram[d * i:d * (i + 1), d * j:d * (j + 1)] = tmp
    
    return gram


In [103]:

def fill_gram_theano(n,d,nn,confs):
    gram = np.zeros((n*d,n*d))
    for i in np.arange(n):
        for j in np.arange(n):
            gram[d*i:d*(i+1),d*j:d*(j+1)] = k2_ff(confs[i],confs[j], 1., 1., 1.)
    return gram

def fill_gram_casadi(n,d,nn,confs):
    gram = np.zeros((n*d,n*d))
    for i in np.arange(n):
        for j in np.arange(n):
            gram[d*i:d*(i+1),d*j:d*(j+1)] = k2_ff_c(confs[i].T, confs[j].T, np.zeros(3), np.zeros(3), 1.)
    return gram

def fill_gram_casadi_comp(n,d,nn,confs):
    gram = np.zeros((n*d,n*d))
    for i in np.arange(n):
        for j in np.arange(n):
            gram[d*i:d*(i+1),d*j:d*(j+1)] = k2_ff_c_comp(confs[i],confs[j], np.zeros(3), np.zeros(3), 1.)
    return gram

In [106]:
# define a benchmarking test on random configurations
%timeit
n = 100
d = 3
nn = M
confs = (np.random.rand(n, nn, d)-0.5)*100

# profiling of the three versions
%timeit fill_gram_theano(n,d,nn,confs)
%timeit fill_gram_casadi(n,d,nn,confs)
# %timeit fill_gram_casadi_comp(n,d,nn,confs)
%timeit fill_gram_python_numba(n,d,nn,confs)
# %timeit fill_gram_python(n,d,nn,confs)

2.72 s ± 744 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
12 s ± 791 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.03 s ± 418 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [105]:
gram1, gram2, gram3 = fill_gram_theano(n,d,nn,confs), fill_gram_casadi(n,d,nn,confs), fill_gram_python_numba(n,d,nn,confs)
np.allclose(gram1, gram2) and np.allclose(gram2, gram3)

True

In [68]:
# check that results are identical

print(k2_ff(confs[0],confs[1], 1., 100000., 1.))
print(k2_ff_c(rho1 = confs[0], rho2 = confs[1], r1 = np.zeros(3), r2 = np.zeros(3), sig = 1.))
print(k2_ff_c_comp(rho1 = confs[0], rho2 = confs[1], r1 = np.zeros(3), r2 = np.zeros(3), sig = 1.))


[[-0.45500515  1.92266694 -0.06642014]
 [ 0.61799601 -3.00530171  1.44789122]
 [ 0.23127827 -0.70517407 -0.43742165]]
{'k_h': DM(
[[-0.455005, 1.92267, -0.0664201], 
 [0.617996, -3.0053, 1.44789], 
 [0.231278, -0.705174, -0.437422]])}
{'k_h': DM(
[[-0.455005, 1.92267, -0.0664201], 
 [0.617996, -3.0053, 1.44789], 
 [0.231278, -0.705174, -0.437422]])}


Theano is sensibly faster than the others

## Three Body

In [ ]:
# TO BE FINISHED

In [ ]:
def compile_threebody_singlespecies():
	"""
	This function generates theano compiled kernels for energy and force learning

	The position of the atoms relative to the centrla one, and their chemical species
	are defined by a matrix of dimension Mx5

	Returns:
		k3_ee (func): energy-energy kernel
		k3_ef (func): energy-force kernel
		k3_ff (func): force-force kernel
	"""

	print("Started compilation of theano three body single species kernels")

	# --------------------------------------------------
	# INITIAL DEFINITIONS
	# --------------------------------------------------

	# positions of central atoms
	r1, r2 = T.dvectors('r1d', 'r2d')
	# positions of neighbours
	rho1, rho2 = T.dmatrices('rho1', 'rho2')
	# hyperparameter
	sig = T.dscalar('sig')
	# cutoff hyperparameters
	theta = T.dscalar('theta')
	rc = T.dscalar('rc')

	# positions of neighbours without chemical species

	#rho1s = rho1[:, 0:3]
	#rho2s = rho2[:, 0:3]

	# --------------------------------------------------
	# RELATIVE DISTANCES TO CENTRAL VECTOR AND BETWEEN NEIGHBOURS
	# --------------------------------------------------

	# first and second configuration
	r1j = T.sqrt(T.sum((rho1s[:, :] - r1[None, :]) ** 2, axis=1))
	r2m = T.sqrt(T.sum((rho2s[:, :] - r2[None, :]) ** 2, axis=1))
	rjk = T.sqrt(T.sum((rho1s[None, :, :] - rho1s[:, None, :]) ** 2, axis=2))
	rmn = T.sqrt(T.sum((rho2s[None, :, :] - rho2s[:, None, :]) ** 2, axis=2))

	# --------------------------------------------------
	# BUILD THE KERNEL
	# --------------------------------------------------

	# Squared exp of differences
	se_1j2m = T.exp(-(r1j[:, None] - r2m[None, :]) ** 2 / (2 * sig ** 2))
	se_jkmn = T.exp(-(rjk[:, :, None, None] - rmn[None, None, :, :]) ** 2 / (2 * sig ** 2))
	se_jk2m = T.exp(-(rjk[:, :, None] - r2m[None, None, :]) ** 2 / (2 * sig ** 2))
	se_1jmn = T.exp(-(r1j[:, None, None] - rmn[None, :, :]) ** 2 / (2 * sig ** 2))

	# Kernel not summed (cyclic permutations)
	k1n = (se_1j2m[:, None, :, None] * se_1j2m[None, :, None, :] * se_jkmn)
	k2n = (se_1jmn[:, None, :, :] * se_jk2m[:, :, None, :] * se_1j2m[None, :, :, None])
	k3n = (se_1j2m[:, None, None, :] * se_jk2m[:, :, :, None] * se_1jmn[None, :, :, :])

	# final shape is M1 M1 M2 M2

	ker_jkmn = (k1n + k2n + k3n)

	cut_ik = (T.exp(-theta / T.abs_(rc - r1j[:, None])) *
	          T.exp(-theta / T.abs_(rc - r1j[None, :])) *
	          T.exp(-theta / T.abs_(rc - rjk[:, :])) *
	          (0.5 * (T.sgn(rc - r1j) + 1))[None, :] *
	          (0.5 * (T.sgn(rc - r1j) + 1))[:, None] *
	          (0.5 * (T.sgn(rc - rjk) + 1))[:, :])

	cut_mn = (T.exp(-theta / T.abs_(rc - r2m[:, None])) *
	          T.exp(-theta / T.abs_(rc - r2m[None, :])) *
	          T.exp(-theta / T.abs_(rc - rmn[:, :])) *
	          (0.5 * (T.sgn(rc - r2m) + 1))[None, :] *
	          (0.5 * (T.sgn(rc - r2m) + 1))[:, None] *
	          (0.5 * (T.sgn(rc - rmn) + 1))[:, :])

	ker_jkmn_withcutoff = ker_jkmn * cut_ik[:, :, None, None] * cut_mn[None, None, :, :]

	# --------------------------------------------------
	# REMOVE DIAGONAL ELEMENTS
	# --------------------------------------------------

	mask_jk = T.ones_like(rjk) - T.identity_like(rjk)
	mask_mn = T.ones_like(rmn) - T.identity_like(rmn)

	mask_jkmn = mask_jk[:, :, None, None] * mask_mn[None, None, :, :]

	k_cutoff = T.sum(ker_jkmn_withcutoff * mask_jkmn)

	# --------------------------------------------------
	# FINAL FUNCTIONS
	# --------------------------------------------------

	# energy energy kernel
	k_ee_fun = function([r1, r2, rho1, rho2, sig, theta, rc], k_cutoff, on_unused_input='warn')

	# energy force kernel
	k_ef_cut = T.grad(k_cutoff, r2)
	k_ef_fun = function([r1, r2, rho1, rho2, sig, theta, rc], k_ef_cut, on_unused_input='warn')

	# force force kernel
	k_ff_cut = T.grad(k_cutoff, r1)
	k_ff_cut_der, updates = scan(lambda j, k_ff_cut, r2: T.grad(k_ff_cut[j], r2),
	                             sequences=T.arange(k_ff_cut.shape[0]), non_sequences=[k_ff_cut, r2])
	k_ff_fun = function([r1, r2, rho1, rho2, sig, theta, rc], k_ff_cut_der, on_unused_input='warn')

	# WRAPPERS (we don't want to plug the position of the central element every time)

	def k3_ee(conf1, conf2, sig, theta, rc):
		"""
		Two body kernel for energy-energy correlation

		Args:
			conf1: first configuration.
			conf2: second configuration.
			sig: lengthscale hyperparameter.
			theta: cutoff smoothness hyperparameter.
			rc: cutoff distance hyperparameter.

		Returns:
			kernel (scalar):

		"""
		return k_ee_fun(np.zeros(3), np.zeros(3), conf1, conf2, sig, theta, rc)

	def k3_ef(conf1, conf2, sig, theta, rc):
		"""
		Two body kernel for energy-force correlation

		Args:
			conf1: first configuration.
			conf2: second configuration.
			sig: lengthscale hyperparameter.
			theta: cutoff smoothness hyperparameter.
			rc: cutoff distance hyperparameter.

		Returns:
			kernel (vector):
		"""

		return k_ef_fun(np.zeros(3), np.zeros(3), conf1, conf2, sig, theta, rc)

	def k3_ff(conf1, conf2, sig, theta, rc):
		"""
		Two body kernel for force-force correlation

		Args:
			conf1: first configuration.
			conf2: second configuration.
			sig: lengthscale hyperparameter.
			theta: cutoff smoothness hyperparameter.
			rc: cutoff distance hyperparameter.

		Returns:
			kernel (matrix):
		"""

		return k_ff_fun(np.zeros(3), np.zeros(3), conf1, conf2, sig, theta, rc)

	print("Ended compilation of theano three body single species kernels")

	return k3_ee, k3_ef, k3_ff

In [ ]:
k3_ee, k3_ef, k3_ff = compile_twobody_singlespecies()

In [ ]:
# --------------------------------------------------
# INITIAL DEFINITIONS
# --------------------------------------------------

# positions of central atoms
M = 3
r1, r2 = SX.sym( 'r1', 1, 3  ), SX.sym( 'r2', 1, 3 )
d1, d2 = SX.sym( 'd1'  ), SX.sym( 'd2')
rho1, rho2 = SX.sym('rho1', M, 3 ), SX.sym('rho2', M, 3 )
sig = SX.sym('sig')
#theta = SX.sym('theta')
#rc = SX.sym('rc')
#rho1, rho2 = MX.sym('rho1', 3 ), MX.sym( 'rho2')


                

In [ ]:
k3 = SX.zeros()
for i1 in range(M):
    for j1 in range(M):
        for i2 in range(M):
            for j2 in range(M):
                
                ri, rj, rij = norm_2(rho1[i, :] - r1), norm_2(rho1[j, :] - r1), norm_2(rho1[i, :] - rho1[j, :])
                ri2, rj2, ri2j2 = norm_2(rho2[i2, :] - r2), norm_2(rho2[j2, :] - r2), norm_2(rho2[i2, :] - rho2[j2, :])
                
                k3 = k3 + exp(-(ri - ri2)**2/(2*sig**2)) * \
                    exp(-(rj - rj2)**2/(2*sig**2)) * \
                    exp(-(rij - ri2j2)**2/(2*sig**2))
                

In [ ]:
grad3 = gradient(k3, r1)
hess3 = jacobian( grad , r2)

In [ ]:
k3_ee_c = Function('k2_ee_c', [rho1, rho2, r1, r2, sig], [k3], ['rho1', 'rho2', 'r1', 'r2', 'sig'] , [ 'k3' ] )
k3_ef_c = Function('k2_ef_c', [rho1, rho2, r1, r2, sig], [grad3], ['rho1', 'rho2', 'r1', 'r2', 'sig'] , [ 'k3_g' ] )
k3_ff_c = Function('k2_ff_c', [rho1, rho2, r1, r2, sig], [hess3], ['rho1', 'rho2', 'r1', 'r2', 'sig'] , [ 'k3_h' ] )

In [ ]:
def fill_gram_theano3():
    gram = np.zeros((n*d,n*d))
    for i in np.arange(n):
        for j in np.arange(n):
            gram[d*i:d*(i+1),d*j:d*(j+1)] = k3_ff(confs[i],confs[j], 1., 1., 1.)
    return gram

def fill_gram_casadi3():
    gram = np.zeros((n*d,n*d))
    for i in np.arange(n):
        for j in np.arange(n):
            gram[d*i:d*(i+1),d*j:d*(j+1)] = k3_ff_c(confs[i], confs[j], np.zeros(3), np.zeros(3), 1.)
    return gram

In [ ]:
%timeit fill_gram_theano3()

In [ ]:
%timeit fill_gram_casadi3()